<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [2]:
import pandas as pd
import psycopg2
import warnings

In [3]:
# параметры подключения
DBNAME = 
USER = 
PASSWORD = 
HOST = 
PORT = 

In [4]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

warnings.filterwarnings('ignore')

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [5]:
query_3_1 = f'''
        SELECT 
            COUNT(id) AS vacancies_count
        
        FROM
            VACANCIES
'''

In [6]:
vacancies_count = pd.read_sql_query(query_3_1, connection)
display(vacancies_count)

,vacancies_count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [7]:
query_3_2 = f'''
        SELECT
            COUNT(id) AS employers_count
        
        FROM
            EMPLOYERS
'''

In [8]:
employers_count = pd.read_sql_query(query_3_2, connection)
display(employers_count)

,employers_count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [9]:
query_3_3 = f''' 
        SELECT
            COUNT(id) AS areas_count

        FROM
            AREAS
'''

In [10]:
areas_count = pd.read_sql_query(query_3_3, connection)
display(areas_count)

,areas_count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [11]:
query_3_4 = f'''
        SELECT
            COUNT(id) AS industries_count

        FROM
            INDUSTRIES
'''

In [12]:
industries_count = pd.read_sql_query(query_3_4, connection)
display(industries_count)

,industries_count
0,294


***

> ### Выводы по предварительному анализу данных: 
>
> - Исследуемый датасет представляет собой объёмный набор данных, в котором содержиться информация о кандидатах, работодателях, их местоположении и сфере деятельности;
> - Набор данных имеет широкую географию городов и сфер деятельности;
> - В среднем на одного работодателя приходится по две вакансии, а на каждую область около 38 вакансий.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [13]:
query_4_1 = f'''
        SELECT
            COUNT(v.id) AS cnt,
            a.name AS area

        FROM
            VACANCIES as v
        LEFT JOIN
            AREAS AS a ON v.area_id = a.id

        GROUP BY
            a.name

        ORDER BY
            COUNT(v.id) DESC

        LIMIT
            5        
'''

In [14]:
top_vacancies = pd.read_sql_query(query_4_1, connection)
display(top_vacancies)

,cnt,area
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [15]:
query_4_2 = f'''
        SELECT
            COUNT(id) AS not_null_salary
        
        FROM
            VACANCIES
        
        WHERE
            salary_from IS NOT NULL OR salary_to IS NOT NULL
'''

In [16]:
not_null_salary = pd.read_sql_query(query_4_2, connection)
display(not_null_salary)

,not_null_salary
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [17]:
query_4_3 = f'''
        SELECT
            INT4(AVG(salary_from)) AS avg_salary_from,
            INT4(AVG(salary_to)) AS avg_salary_to

        FROM
            VACANCIES
'''

In [18]:
salary_from_to = pd.read_sql_query(query_4_3, connection)
display(salary_from_to)

,avg_salary_from,avg_salary_to
0,71065,110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [19]:
query_4_4 = f'''
        SELECT
            COUNT(id) AS count,
            schedule,
            employment

        FROM
            VACANCIES

        GROUP BY
            schedule, employment
        
        ORDER BY
            COUNT(id) DESC
        
        LIMIT
            3
'''

In [20]:
top_schedule_employment = pd.read_sql_query(query_4_4, connection)
display(top_schedule_employment)

,count,schedule,employment
0,35367,Полный день,Полная занятость
1,7802,Удаленная работа,Полная занятость
2,1593,Гибкий график,Полная занятость


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [21]:
query_4_5 = f'''
        SELECT
            COUNT(id),
            experience

        FROM
            VACANCIES

        GROUP BY
            experience

        ORDER BY
            COUNT(id)
'''

In [22]:
experience = pd.read_sql_query(query_4_5, connection)
display(experience)

,count,experience
0,1337,Более 6 лет
1,7197,Нет опыта
2,14511,От 3 до 6 лет
3,26152,От 1 года до 3 лет


***

> ### Выводы по детальному анализу вакансий:
>
> - Наибольшее количество вакансий (топ-5) расположено в городах федерального/республиканского значения своих стран, или же просто в наикрупнейших городах России, Белоруссии и Казахстана. Данный факт не является удивительным, так как большинство фирм старается иметь представительства в крупных городах, для престижа и привлечения большего количества клиентов. Также крупные города имеют более развитую инфрасткуру, что привлекает большее количество людей и создаёт здоровую конкуренцию среди квалифицированных специалистов;
> - Больше половины работодателей не указывают зарплатную вилку, однако те работадатели, которые указывают зарплатную вилку - предлагают неплохие деньги ([в два и более раза больше медианных значений по России](https://gogov.ru/articles/median-salary)). Те, кто не указывают зарплатную вилку вероятно надеятся установить зарплату по результатам собеседования и конкурсного отбора;
> - Большая часть работодателей ищёт молодых специалистов с небольшим опытом на стандартную занятость 5/2. Вероятно это связано с тем, что молодых специалистов проще обучить специфике работы, интегрировать их в свою фирму и установить им зарплату более выгодную организации. Также привлечение более опытных специалистов на более отвественные должности нередко осуществляется через знакомства и рекомендации.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [23]:
query_5_1 = f'''
        SELECT
            e.name,
            COUNT(v.id)

        FROM
            VACANCIES AS v
        LEFT JOIN
            EMPLOYERS AS e ON v.employer_id = e.id

        GROUP BY
            e.name
        
        ORDER BY
            COUNT(v.id) DESC

        LIMIT
            5
'''

In [24]:
top_employers = pd.read_sql_query(query_5_1, connection)
display(top_employers)

,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [25]:
query_5_2 = '''
        SELECT
            a.name,
            COUNT(DISTINCT e.id) AS "Count employers",
            COUNT(DISTINCT v.id) AS "Count vacancies" 

        FROM
            AREAS AS a
        LEFT JOIN
            EMPLOYERS AS e ON a.id = e.area
        LEFT JOIN
            VACANCIES AS v ON v.area_id = a.id

        WHERE
            v.id IS NULL

        GROUP BY
            a.id

        ORDER BY
            COUNT(e.id) DESC

        LIMIT 
            3 
'''

In [26]:
top_areas_employers = pd.read_sql_query(query_5_2, connection)
display(top_areas_employers)

,name,Count employers,Count vacancies
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [27]:
query_5_3 = f'''
        SELECT
            e.name,
            COUNT(DISTINCT a.id) AS "Count areas" 

        FROM
            VACANCIES AS v
        LEFT JOIN
            AREAS AS a ON a.id = v.area_id
        LEFT JOIN
            EMPLOYERS AS e ON e.id = employer_id

        GROUP BY
            e.id
        
        ORDER BY
            COUNT(DISTINCT a.id) DESC

        LIMIT
            3
'''

In [28]:
employers_area = pd.read_sql_query(query_5_3, connection)
display(employers_area)

,name,Count areas
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [29]:
query_5_4 = f'''
        SELECT
            COUNT(e.id) AS "Count employers without industries"

        FROM
            EMPLOYERS AS e
        LEFT JOIN
            EMPLOYERS_INDUSTRIES AS ei ON e.id = ei.employer_id
        LEFT JOIN
            INDUSTRIES AS i ON ei.industry_id = i.id

        WHERE
            ei.industry_id IS NULL
'''

In [30]:
emp_without_ind = pd.read_sql_query(query_5_4, connection)
display(emp_without_ind)

,Count employers without industries
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [31]:
query_5_5 = f'''
        SELECT
            e.name
        
        FROM
            EMPLOYERS AS e
        LEFT JOIN
            EMPLOYERS_INDUSTRIES AS ei ON e.id = ei.employer_id
        LEFT JOIN
            INDUSTRIES AS i ON ei.industry_id = i.id

        GROUP BY
            e.id

        HAVING
            COUNT(i.id) = 4
        
        ORDER  BY
            e.name

        OFFSET
            2

        LIMIT
            1
'''

In [32]:
third_company_industry = pd.read_sql_query(query_5_5, connection)
display(third_company_industry)

,name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [33]:
query_5_6 = f'''
        SELECT
            COUNT(i.name) AS "count soft employers"

        FROM
            EMPLOYERS AS e
        LEFT JOIN
            EMPLOYERS_INDUSTRIES AS ei ON e.id = ei.employer_id
        LEFT JOIN
            INDUSTRIES AS i ON ei.industry_id = i.id

        WHERE
            i.name = 'Разработка программного обеспечения' 
'''

In [34]:
soft_employers = pd.read_sql_query(query_5_6, connection)
display(soft_employers)

,count soft employers
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [35]:
tables = pd.read_html('https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8')
cities_df = tables[1]
cities_list = tuple(cities_df['Город'])

print(cities_list)

('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Красноярск', 'Челябинск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')


In [36]:
query_5_7 = f'''
        (SELECT
            a.name,
            COUNT(v.id)
            
        FROM
            VACANCIES AS v
        LEFT JOIN
            AREAS AS a ON v.area_id = a.id
        LEFT JOIN
            EMPLOYERS AS e ON v.employer_id = e.id 

        WHERE
            e.name = 'Яндекс' AND a.name IN {cities_list}

        GROUP BY 
            a.name)

        UNION

        (SELECT
            'Total' AS name,
            COUNT(v.id)
        
        FROM
           VACANCIES AS v
        LEFT JOIN
            AREAS AS a ON v.area_id = a.id
        LEFT JOIN
            EMPLOYERS AS e ON v.employer_id = e.id

        WHERE
            e.name = 'Яндекс' AND a.name IN {cities_list})   

        ORDER BY
            count
'''

In [37]:
yandex_cities = pd.read_sql_query(query_5_7, connection)
display(yandex_cities)

,name,count
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

> ### Выводы по анализу работодателей:
> - Самым часто встречающимся работодателем является компания "Яндекс". Вероятно это связано с обширным списком сфер деятельности компании (Электронная почта, поисковая система, такси, каршеринг, доставка еды);
> - Компания "Яндекс" представлена во всех крупных городах России, а также в наибольшем колечестве регионов России. Это даёт повод назвать "Яндекс" практически монополистом в ряде сфер деятельности (доставка еды, такси и интернет-поисковик);
> - Почти каждый пятый работодатель не указывают свою сферу деятельности. Это может быть связано с желанием привлечь большее количество специалистов из смежных областей.
> - Из тех же фирм, что указывают свою сферу деятельности, почти что каждая десятая занимается разработкой программного обеспечения. Данный факт свидетельствует о набирающей обороты цифровизации всех сфер деятельности.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [38]:
query_6_1 = f'''
        SELECT
            COUNT(v.name) AS count_data_vacancies

        FROM
            VACANCIES AS v

        WHERE
             LOWER(v.name) LIKE '%data%' OR LOWER(v.name) LIKE '%данн%'
'''

In [39]:
count_data_vacancies = pd.read_sql_query(query_6_1, connection)
display(count_data_vacancies)

,count_data_vacancies
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [40]:
query_6_2 = f'''
        SELECT
            COUNT(v.id) AS count_junior_dst

        FROM
            VACANCIES AS v

        WHERE
            (LOWER(v.name) LIKE '%data scientist%' OR 
            LOWER(v.name) LIKE '%data science%' OR 
            LOWER(v.name) LIKE '%исследователь данных%' OR 
            (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%') OR
            LOWER(v.name) LIKE '%machine learning%' OR
            LOWER(v.name) LIKE '%машинн%обучен%') 
            AND 
            (LOWER(v.name) LIKE '%junior%' OR 
            v.experience = 'Нет опыта' OR 
            v.employment = 'Стажировка')
'''

In [41]:
cnt_dst_junior = pd.read_sql_query(query_6_2, connection)
display(cnt_dst_junior)

,count_junior_dst
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

**Критерии для отнесения вакансии к DS указаны в предыдущем задании.**

In [42]:
query_6_3 = f'''
        SELECT
            COUNT(v.id) AS count_ds_sql

        FROM
            VACANCIES AS v

        WHERE
            (LOWER(v.name) LIKE '%data scientist%' OR
            LOWER(v.name) LIKE '%data science%' OR
            LOWER(v.name) LIKE '%исследователь данных%' OR
            (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%') OR
            LOWER(v.name) LIKE '%machine learning%' OR
            LOWER(v.name) LIKE '%машинн%обучен%')
            AND
            (v.key_skills LIKE '%PostgreS%' OR
            v.key_skills LIKE '%SQL%')
'''

In [43]:
count_ds_sql = pd.read_sql_query(query_6_3, connection)
display(count_ds_sql)

,count_ds_sql
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

**Это можно сделать помощью запроса, аналогичного предыдущему.**

In [44]:
query_6_4 = f'''
        SELECT
            COUNT(v.id) AS count_ds_python

        FROM
            VACANCIES AS v

        WHERE
            (LOWER(v.name) LIKE '%data scientist%' OR 
            LOWER(v.name) LIKE '%data science%' OR 
            LOWER(v.name) LIKE '%исследователь данных%' OR 
            (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%') OR
            LOWER(v.name) LIKE '%machine learning%' OR
            LOWER(v.name) LIKE '%машинн%обучен%')
            AND
            (v.key_skills LIKE '%Python%')
'''

In [45]:
count_ds_python = pd.read_sql_query(query_6_4, connection)
display(count_ds_python)

,count_ds_python
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [46]:
query_6_5 = f'''
        SELECT
            ROUND(AVG(LENGTH(v.key_skills) - LENGTH(REPLACE(v.key_skills, '\t', '')) + 1), 2)

        FROM
            VACANCIES AS v

        WHERE
            (LOWER(v.name) LIKE '%data scientist%' OR 
            LOWER(v.name) LIKE '%data science%' OR 
            LOWER(v.name) LIKE '%исследователь данных%' OR 
            (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%') OR
            LOWER(v.name) LIKE '%machine learning%' OR
            LOWER(v.name) LIKE '%машинн%обучен%')            
'''

In [47]:
cnt_key_skills = pd.read_sql_query(query_6_5, connection)
display(cnt_key_skills)

,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [50]:
query_6_6 = f'''
        SELECT
            v.experience,
            INT4(ROUND(AVG((COALESCE(v.salary_to, v.salary_from) + COALESCE(v.salary_from, v.salary_to)) / 2))) AS avg_dst_salary

        FROM
            VACANCIES AS v

        WHERE
            ((LOWER(v.name) LIKE '%data scientist%' OR 
            LOWER(v.name) LIKE '%data science%' OR 
            LOWER(v.name) LIKE '%исследователь данных%' OR 
            (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%') OR
            LOWER(v.name) LIKE '%machine learning%' OR
            LOWER(v.name) LIKE '%машинн%обучен%'))
            AND
            (v.salary_from IS NOT NULL OR
            v.salary_to IS NOT NULL)

        GROUP BY
            v.experience
'''

In [51]:
avg_dst_salary = pd.read_sql_query(query_6_6, connection)
display(avg_dst_salary)

,experience,avg_dst_salary
0,Нет опыта,74643
1,От 1 года до 3 лет,139675
2,От 3 до 6 лет,243115


***

>### Выводы по предметному анализу:
> - Доля вакансий, которые имеют отношение к данным занимает всего 3.6%. В современных российских реалиях это может говорить о том, что отрасль качественной автоматизированной обработки данных, находится в самом начале своего пути;
> - Молодому специалисту в области Data science будет довольно-таки непросто найти работу, так как количество вакансий на рынке труда составляет всего 0.1% от общего объёма вакансий;
> - Знание языка программирования Python является более обязательным, чем знание языка программирования для обработки данных SQL. Вероятно это связано с разделением обязанностей среди специалистов по обработке данных и программированию;
> - Для того, чтобы приступить к поиску работы в области Data science, необходимо уверенно овладеть 6 навыками, среди которых знание языка программирования Python более предпочтительно чем знание языка программирования SQL.
> - Специалист Data Science, освоившейся в профессии и приобрёвший опыт работы, может расчитывать на зароботную плату значительно превышающую медианную по России [35.000 рублей](https://gogov.ru/articles/median-salary)

# Дополнительные исследования

In [ ]:
query_7_1 = f'''
        SELECT
            INT4(ROUND(AVG((COALESCE(v.salary_to, v.salary_from) + COALESCE(v.salary_from, v.salary_to)) / 2))) AS avg_salary

        FROM
            VACANCIES AS v
'''
avg_salary = pd.read_sql_query(query_7_1, connection)
display(avg_salary)

,avg_salary
0,84315


> ### Выводы:
> Умение пользоваться компьютером (в частности сайтом [HeadHunter](https://hh.ru/)) является явным преимуществом, так как средняя предлагаемая зарплата по данным наших исследований на [50% больше средней зароботной платы по России.](https://gorodrabot.ru/salary) 

In [ ]:
query_7_2 = f'''
        SELECT
            a.name AS area,
            INT4(ROUND(AVG((COALESCE(v.salary_to, v.salary_from) + COALESCE(v.salary_from, v.salary_to)) / 2))) AS avg_area_salary

        FROM
            VACANCIES AS v
        LEFT JOIN
            AREAS AS a ON v.area_id = a.id

        WHERE
            v.salary_from IS NOT NULL OR
            v.salary_to IS NOT NULL
        
        GROUP BY
            a.id

        ORDER BY
            avg_area_salary DESC

        LIMIT
            5
'''
avg_area_salary = pd.read_sql_query(query_7_2, connection)
display(avg_area_salary)

,area,avg_area_salary
0,Германия,452751
1,Болгария,324348
2,Испания,300000
3,Черногория,288589
4,Литва,263019


>### Выводы:
>Лидерами по зароботной плате по-прежнему остаются работодатели из Европы

In [ ]:
query_7_3 = f'''
        SELECT
            a.name AS area,
            INT4(ROUND(AVG((COALESCE(v.salary_to, v.salary_from) + COALESCE(v.salary_from, v.salary_to)) / 2))) AS avg_area_salary

        FROM
            VACANCIES AS v
        LEFT JOIN
            AREAS AS a ON v.area_id = a.id

        WHERE
            (v.salary_from IS NOT NULL OR
            v.salary_to IS NOT NULL)
            AND
            (LOWER(v.name) LIKE '%data scientist%' OR 
            LOWER(v.name) LIKE '%data science%' OR 
            LOWER(v.name) LIKE '%исследователь данных%' OR 
            (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%') OR
            LOWER(v.name) LIKE '%machine learning%' OR
            LOWER(v.name) LIKE '%машинн%обучен%')
        
        GROUP BY
            a.id

        ORDER BY
            avg_area_salary DESC

        LIMIT
            5
'''
avg_dst_area_salary = pd.read_sql_query(query_7_3, connection)
display(avg_dst_area_salary)

,area,avg_area_salary
0,Кипр,300000
1,Армения,268863
2,Турция,233794
3,Сербия,233794
4,Черногория,233794


>### Выводы:
>Заграничные работодатели, заинтересованные в специалистах Data Science, также готовы предоставить более выгодные условия. Аргумент о более дорогой заграничной жизни имеет слабый успех, так как специалисты IT чаще остальных имеет возможность работать удалённо из любой точки мира.

In [ ]:
query_7_4 = f'''
        SELECT
            i.name AS industry,
            INT4(ROUND(AVG((COALESCE(v.salary_to, v.salary_from) + COALESCE(v.salary_from, v.salary_to)) / 2))) AS avg_industry_salary

        FROM
            VACANCIES AS v
        LEFT JOIN
            EMPLOYERS_INDUSTRIES AS ei ON v.employer_id = ei.employer_id
        LEFT JOIN
            INDUSTRIES AS i ON ei.industry_id = i.id

        WHERE 
            v.salary_from IS NOT NULL OR
            v.salary_to IS NOT NULL

        GROUP BY
            i.name

        ORDER BY
            avg_industry_salary DESC

        LIMIT
            5
'''

avg_industry_salary = pd.read_sql_query(query_7_4, connection)
display(avg_industry_salary)

,industry,avg_industry_salary
0,"Агрохимия (продвижение, оптовая торговля)",381429
1,Агрохимия (производство),339875
2,Игорный бизнес,191905
3,Корма для животных (производство),168300
4,Зоомагазин,167613


>### Выводы:
>Среди сфер деятельности, самые щедрые работодатели оказались в области животноводства и земледелия. Данный фактор в очередной раз доказывает, что пока люди едят и пьют, в области сельского хозяйства всегда будет рост, развитие и приток капитала.

In [ ]:
query_7_5 = f'''
        SELECT
            COUNT(v.id) AS count_ds

        FROM
            VACANCIES AS v

        WHERE
            LOWER(v.name) LIKE '%data scientist%' OR 
            LOWER(v.name) LIKE '%data science%' OR 
            LOWER(v.name) LIKE '%исследователь данных%' OR 
            (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%') OR
            LOWER(v.name) LIKE '%machine learning%' OR
            LOWER(v.name) LIKE '%машинн%обучен%'
'''

count_ds = pd.read_sql_query(query_7_5, connection)
display(count_ds)

,count_ds
0,480


> ### Выводы:
> - Вакансий, которые "ищут" специаистов в области Data Science, всего 480, однакого это помогает нам сделать ряд более точных выводов:
>   - если в общей массе специалистов Data Science ищет лишь каждый 100 работодатель, то среди работодателей, имеющих отношение к данным, специалистов Data science ищет уже почти что каждый 3;
>   - доля "новичков" среди специалистов Data Science составляет уже почти 11%, вместо 3% среди специалистов, занимающимися работой с данными;
>   - знание языка программирования Python является практически обязательным для специалиста Data science, так как встречается в 73% вакансий. Что нельзя сказать про язык программирования SQL, так как он встречается лишь в 40% вакансий. 

In [ ]:
query_7_6 = f'''
        SELECT
            i.name AS industry,
            COUNT(v.id) AS vacancies_count

        FROM
            VACANCIES AS v
        LEFT JOIN
            EMPLOYERS_INDUSTRIES AS ei ON v.employer_id = ei.employer_id
        LEFT JOIN
            INDUSTRIES AS i ON ei.industry_id = i.id

        WHERE
            i.name IS NOT NULL

        GROUP BY
            i.id

        ORDER BY
            COUNT(v.id) DESC

        LIMIT
            5
'''

count_industry = pd.read_sql_query(query_7_6, connection)
display(count_industry)

,industry,vacancies_count
0,Разработка программного обеспечения,12499
1,"Системная интеграция, автоматизации технологи...",11034
2,"Интернет-компания (поисковики, платежные систе...",6413
3,Банк,2742
4,Мобильная связь,1585


>### Выводы:
>наибольшее количество вакансий (ТОП-5, а точнее ~70%) сконцентрированно в области цифровых технологий. Это свидетельствует о том, что цифровизация затрагивает практически все области жизнедеятельности человека.

># Общие выводы по проекту:
>- Данный проект представляет собой объёмный дата-сет, состоящий из 5 таблиц, описывающих ~49.000 вакансий от ~23.500 работодателей в более чем 1.300 регионах не только России, но и зарубежья;
>- Наибольшее количество вакансий сконцентрировано в городах федерального значения и в столицах. В таких городах более развита инфрастуктура и проживает большее количество людей. Большинство компаний старается расположить предаставительства в региональных центрах;
>- Большинство работодателей ищёт сотрудников на классические "полный день и полная занятость", что является наиболее оптимальным режимом работы для большинства фирм по сей день;
>- Больше половины работодателей ищет сотрудников с опытом работы от 1 до 3 лет. Вероятно это связяно с тем, что таких сотрудников проще интегрировать в компанию. Людей же на более ответственные должности могут искать через знакомства и рекомендации;
>- Яндекс лидирует по количеству вакансий и количеству регионов, где компания ищет сотрудников. Данный факт свидетельсвует о бурном ростей компании и её всестороннем развии; 
>- Из тех работодателей, что указывают свою сферу деятельности, почти каждый 10 занимается разработкой программного обеспечения. Данный факт позволяет отметить положительную тендецию цифровизации всех сфер деятельности, развитие комьютерных технологий и попытку различных фирм лично автоматизировать производственные процессы;
>- Специализация Data Science является весьма прорывной и полезной, однако количество вакансий в общей доле составляет буквально 1%. Количество вакансий для начинающих специлистов оставляет желать лучшего. Однако, если анализировать вакансии в разрезе работодателей, которые имеют отношение к данным или непосредственно к Data Science, можно сделать более благоприятные прогнозы;
>- Основными требованиями к специлистам Data Science являются: язык программирования Python и SQL/PostgreSQL, которые по факту являются основными "орудиями труда" специалиста;
>- Опытный специалист в области Data Science может предентовать на зароботную плату значительно превышающую среднюю и медианную зарплаты в России.